# Eyepoint_cataract_trends

Customer is preparing for the commercial launch of two ophthalmic therapies:
1. DEXYCU: A dropless long-acting therapeutic for treating inflammation associated with cataract surgery.
2. YUTIQ: A long-acting therapeutic delivered via the Durasert bioerodible technology platform for treatment of posterior segment uveitis.

To support the upcoming launch, Customer is seeking a data and analytics partner to provide robust insights into the target
patient and provider populations to support salesforce optimization and market access priorities.

# Connections

## Importing required modules

In [3]:
## importing packages and setting connector to snowflake
import pandas as pd
from drg_connect import Snowflake
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings('ignore')

## Snowflake connection parameter

In [4]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW', warehouse = 'RWD_WH_ANALYTICS' ,database='PROJECT_ANALYTICS',schema ='EYEPOINT_PHYFACTAR')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# RAVEN PROCEDURE
Getting data from procedure table based on procedure codes of interest

In [4]:
%%read_sql
DROP TABLE IF EXISTS Eyepoint_cataract_raven_med_trends_1;
CREATE TABLE Eyepoint_cataract_raven_med_trends_1 as
      SELECT CLAIM_NUMBER,
             ENCRYPTED_KEY_1 as patient_suffix,
             ENCRYPTED_KEY_2,
             case when ENCRYPTED_KEY_2 is null then ENCRYPTED_KEY_1 else ENCRYPTED_KEY_1||ENCRYPTED_KEY_2 end as Patient_ID,
             PROCEDURE,
             YEAR_OF_SERVICE,
             PLACE_SERVICE,
             NDC
        FROM RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE
       WHERE PROCEDURE IN ('66940','66920','66982','66983','66984','G9389',
                           'G9390','66840','66850','66852','65175','66986',
                           '13.71','13.41','13.42','13.43','13.6','13.69',
                           '1371','1341','1342','1343','136','1369')
             AND YEAR_OF_SERVICE >= '2015-01-01'
             AND YEAR_OF_SERVICE <= '2018-05-31'
             AND encrypted_key_1 IS NOT NULL 
             AND upper(encrypted_key_1) <> 'NULL'
             AND encrypted_key_1 <> '' 
             AND left(encrypted_key_1,3) <> 'XXX';

Query started at 09:05:18 PM India Standard Time; Query executed in 0.06 mQuery started at 09:05:22 PM India Standard Time; Query executed in 5.75 m

status
0  Table EYEPOINT_CATARACT_RAVEN_MED_TRENDS_1 suc...

Getting counts of patients, claim no. on the basis of procedures

In [3]:
%%read_sql
  SELECT COUNT(*), 
         COUNT(distinct patient_id) as pat_cnt, 
         COUNT(distinct claim_number) as clm_cnt
    FROM Eyepoint_cataract_raven_med_trends_1

Query started at 12:29:35 PM India Standard TimeInitiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 0.22 m

COUNT(*)  pat_cnt  clm_cnt
0   7924666  3099035  7765652

In [4]:
%%read_sql breakup
  SELECT year(YEAR_OF_SERVICE) as year,
         COUNT(*),
         COUNT(distinct patient_id) as pat_cnt, 
         COUNT(distinct claim_number) as clm_cnt
    FROM Eyepoint_cataract_raven_med_trends_1
   GROUP BY year
   ORDER BY year;

Query started at 12:30:04 PM India Standard Time; Query executed in 0.06 m

year  COUNT(*)  pat_cnt  clm_cnt
0  2015   2308155   953620  2276765
1  2016   2365197   971345  2334878
2  2017   2285148   949638  2251212
3  2018    966166   404668   902954

# RAVEN PAYER
Pulling data from payer table to exclude medicare part from Raven

In [5]:
%%read_sql
DROP TABLE IF EXISTS EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_1;
CREATE TABLE EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_1 as
    SELECT A.*,
           B.PAYER_ID,
           B.TYPE_COVERAGE
  FROM Eyepoint_cataract_raven_med_trends_1 A
       LEFT JOIN RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PAYER B
              ON A.claim_number = B.claim_number;

Query started at 12:30:33 PM India Standard Time; Query executed in 0.04 mQuery started at 12:30:36 PM India Standard Time; Query executed in 1.68 m

status
0  Table EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PA...

## Type of coverage mapping

In [6]:
%%read_sql
DROP TABLE IF EXISTS EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_2;
CREATE TABLE EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_2 as
      SELECT A.*,
             C.MAP1 as TOC
        FROM EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_1 A
             LEFT Join project_analytics.Pfizer.Plan_Type_Mapping_Options C
                    on A.TYPE_COVERAGE = C.TYPE_OF_COVERAGE;

Query started at 12:34:08 PM India Standard Time; Query executed in 0.05 mQuery started at 12:34:11 PM India Standard Time; Query executed in 0.12 m

status
0  Table EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PA...

In [7]:
%%read_sql
    SELECT count(*),
           count(distinct patient_id) as pat_cnt,
           count(distinct claim_number) as cml_cnt
      FROM EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_2;

Query started at 12:38:39 PM India Standard Time; Query executed in 0.11 m

COUNT(*)  pat_cnt  cml_cnt
0   9893723  3099035  7765652

Checking counts of patients and claims per toc

In [8]:
%%read_sql
    SELECT TOC,
           COUNT(DISTINCT PATIENT_ID) AS PAT_CNT,
           COUNT(DISTINCT CLAIM_NUMBER) AS CLM_CNT
      FROM EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_2
     GROUP BY TOC
     ORDER BY CLM_CNT DESC;

Query started at 12:39:12 PM India Standard Time; Query executed in 0.06 m

toc  pat_cnt  clm_cnt
0          Commercial  2016111  4450433
1        Medicare FFS  1463399  3591015
2  Medicare Advantage   169444   537723
3            Medicaid   160104   323465
4              Others   169924   301363
5                None   130711   214876

In [9]:
%%read_sql  raven_trends_1
  SELECT TOC,
         year(YEAR_OF_SERVICE) as year,
         COUNT(distinct patient_id) as pat_cnt, 
         COUNT(distinct claim_number) as clm_cnt
    FROM EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_2
   GROUP BY TOC,year
   ORDER BY TOC,year;

Query started at 12:39:48 PM India Standard Time; Query executed in 0.05 m

toc  year  pat_cnt  clm_cnt
0           Commercial  2015   622236  1314856
1           Commercial  2016   624603  1316632
2           Commercial  2017   608229  1279296
3           Commercial  2018   268364   539720
4             Medicaid  2015    51066    98400
5             Medicaid  2016    51390    99405
6             Medicaid  2017    49078    94263
7             Medicaid  2018    17379    31398
8   Medicare Advantage  2015    41423   124799
9   Medicare Advantage  2016    54868   173132
10  Medicare Advantage  2017    57759   178422
11  Medicare Advantage  2018    23074    61372
12        Medicare FFS  2015   466669  1082453
13        Medicare FFS  2016   478935  1131845
14        Medicare FFS  2017   440902  1029040
15        Medicare FFS  2018   159737   347766
16              Others  2015    41241    70531
17              Others  2016    49654    83862
18              Others  2017    63317   107144
19              Others  2018    23952    39830
20                None  2015    46851    73774
21                None  2016    37729    62087
22                None  2017    35911    56617
23                None  2018    15257    22399

In [10]:
raven_trends_t = raven_trends_1.pivot_table(index = ['toc'],
                              columns = ['year'],
                              values = ['pat_cnt','clm_cnt'],
                              fill_value= 0)
raven_trends_t

clm_cnt                           pat_cnt                  \
year                   2015     2016     2017    2018    2015    2016    2017   
toc                                                                             
Commercial          1314856  1316632  1279296  539720  622236  624603  608229   
Medicaid              98400    99405    94263   31398   51066   51390   49078   
Medicare Advantage   124799   173132   178422   61372   41423   54868   57759   
Medicare FFS        1082453  1131845  1029040  347766  466669  478935  440902   
Others                70531    83862   107144   39830   41241   49654   63317   

                            
year                  2018  
toc                         
Commercial          268364  
Medicaid             17379  
Medicare Advantage   23074  
Medicare FFS        159737  
Others               23952

In [11]:
# Removing levels of pivoted table
raven_trends_df = pd.DataFrame(raven_trends_t).copy()
raven_trends_df.columns = ['_'.join(reversed(tuple(map(str, t)))) for t in raven_trends_t.columns.values]
raven_trends_df = raven_trends_df.sort_index(axis=1)
raven_trends_df = raven_trends_df.reset_index()
raven_trends_df.head()

toc  2015_clm_cnt  2015_pat_cnt  2016_clm_cnt  2016_pat_cnt  \
0          Commercial       1314856        622236       1316632        624603   
1            Medicaid         98400         51066         99405         51390   
2  Medicare Advantage        124799         41423        173132         54868   
3        Medicare FFS       1082453        466669       1131845        478935   
4              Others         70531         41241         83862         49654   

   2017_clm_cnt  2017_pat_cnt  2018_clm_cnt  2018_pat_cnt  
0       1279296        608229        539720        268364  
1         94263         49078         31398         17379  
2        178422         57759         61372         23074  
3       1029040        440902        347766        159737  
4        107144         63317         39830         23952

In [12]:
raven_trends_df.to_excel('raven_trends_2108.xlsx',index=False)

# Commercial and med. adv. combined

In [14]:
%%read_sql

CREATE TABLE EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_3 as
      SELECT *,
             case when lower(TOC) in ('commercial','medicare advantage') then 'comm_ma'
                  else TOC 
              end as TOC1
        FROM EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_2

Query started at 01:08:49 PM India Standard Time; Query executed in 0.15 m

status
0  Table EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PA...

In [16]:
%%read_sql

SELECT * FROM EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_3  LIMIT 10

Query started at 01:13:13 PM India Standard Time; Query executed in 0.07 m

claim_number  \
0                             vul_110704362   
1  vul_17cd306b-cab3-4f24-acf3-dc232f6ae9f7   
2                        124532000000114529   
3                     con_EI052518251217353   
4  vul_8556bdf6-1cee-45ae-af91-de08aac842f7   
5                             vul_114045455   
6                            alb_1377022827   
7                            alb_1377022827   
8  vul_803ca39d-9487-4e65-9b39-9fb3e899efe6   
9                     con_EI121615227500831   

                                 patient_suffix  \
0  tvz+BkyNNN73OH5l1kI+IP9osoh9UGk8rAM8S4ah2Ls=   
1  5IFkDEkAE2DpJyYPKhCvtUyC5NKpRx+iukt87U6j02c=   
2  HjD8rERlVxixn7i6Ce1aW4Sa4Z2wr6emiT8PHrR3EPI=   
3  2+RoYLm0DPSBOA6ZIrlsnMKKf/IrGLmuzIu1cYMxs/Q=   
4  GN3CkRHKWKlFGBSwgVVBQFxdf9uVq/GGswJvrq3btaM=   
5  xbC07fsvaZBt+ZVaxnIzgu7vEGYzoFnXWmcB27MI+DQ=   
6  UnNncrLHB3fadYy9IxHgy5y/JbP8b+jqD+RzRpHOWHY=   
7  UnNncrLHB3fadYy9IxHgy5y/JbP8b+jqD+RzRpHOWHY=   
8  y9nHD8wrraxji0Cgm3XKsntkG2i3cM2abzD3z6oK5zU=   
9  v8c5rsaYPE4X7fBFVc8plGlXRaw3sEUvstCt4tzLLB0=   

                                encrypted_key_2  \
0  BeWxOkQhiKPdQ9aHfNV+CnTWtWiQ+q7dsfQxV9+ViDY=   
1  KEJwQokCzdsdPkYLZsqZkqDLjFEOWJUuB6LNCgoGc48=   
2  OQUTEc8QMU9Bq6cVsG5Cw++3U2Zw55RHtZD22JNAWeQ=   
3  D8e2dikVDtc8T/AtnL5DwXb7hOwfg4tuxHDCr6/JrJU=   
4  u/PLy9vvHpuZY6tb5lgEF1x63cQ295fLL5S1Lv4PwfA=   
5  cjjaHP34c0NGNNTo8JJaMJufHp4zbvbxnjsVVU8kZas=   
6  kapoaOoUzLsEE+RgVW0gWDXVbezFPi1RF55k2z+YMjs=   
7  kapoaOoUzLsEE+RgVW0gWDXVbezFPi1RF55k2z+YMjs=   
8  52C7va61eNR/pm/cwGt+8vGcZFJa2EVZ1L3tbjT/dqs=   
9  6VyEjt8fzpMY5ImBh2DwyIEuLX8MKfDeyct3Tn6nVzw=   

                                          patient_id procedure  \
0  tvz+BkyNNN73OH5l1kI+IP9osoh9UGk8rAM8S4ah2Ls=Be...     66984   
1  5IFkDEkAE2DpJyYPKhCvtUyC5NKpRx+iukt87U6j02c=KE...     66984   
2  HjD8rERlVxixn7i6Ce1aW4Sa4Z2wr6emiT8PHrR3EPI=OQ...     66984   
3  2+RoYLm0DPSBOA6ZIrlsnMKKf/IrGLmuzIu1cYMxs/Q=D8...     66984   
4  GN3CkRHKWKlFGBSwgVVBQFxdf9uVq/GGswJvrq3btaM=u/...     66984   
5  xbC07fsvaZBt+ZVaxnIzgu7vEGYzoFnXWmcB27MI+DQ=cj...     66984   
6  UnNncrLHB3fadYy9IxHgy5y/JbP8b+jqD+RzRpHOWHY=ka...     66984   
7  UnNncrLHB3fadYy9IxHgy5y/JbP8b+jqD+RzRpHOWHY=ka...     66984   
8  y9nHD8wrraxji0Cgm3XKsntkG2i3cM2abzD3z6oK5zU=52...     66984   
9  v8c5rsaYPE4X7fBFVc8plGlXRaw3sEUvstCt4tzLLB0=6V...     66984   

  year_of_service place_service   ndc  \
0      2016-10-12          None  None   
1      2017-07-27          None  None   
2      2015-06-02            22  None   
3      2018-05-25            83  None   
4      2016-05-24          None  None   
5      2016-12-28          None  None   
6      2016-04-18            24  None   
7      2016-04-18            24  None   
8      2018-01-25          None  None   
9      2015-12-07            13  None   

                                       payer_id type_coverage  \
0  syigx3dkmZiZDnWt2MqqkEgTaW9AQVLIqlpUWoOXgO8=             Z   
1  p1/JQx03T5cafPJnVr9XHxLXdGPLv68e+0QWthE9kLY=            MB   
2                                         03429             8   
3                                         87726            CI   
4  ZwBKdU6Zwyg/Gp6msm1OlVfQuYOfcOXpxBcS2PhzX94=            CI   
5  syigx3dkmZiZDnWt2MqqkEgTaW9AQVLIqlpUWoOXgO8=             Z   
6                                         15202            MB   
7                                         15202            MB   
8  p1/JQx03T5cafPJnVr9XHxLXdGPLv68e+0QWthE9kLY=            MB   
9                                         87726            16   

                  toc          toc1  
0              Others        Others  
1        Medicare FFS  Medicare FFS  
2                None          None  
3          Commercial       comm_ma  
4          Commercial       comm_ma  
5              Others        Others  
6        Medicare FFS  Medicare FFS  
7        Medicare FFS  Medicare FFS  
8        Medicare FFS  Medicare FFS  
9  Medicare Advantage       comm_ma

In [18]:
%%read_sql  raven_trends_comb
  SELECT TOC1,
         year(YEAR_OF_SERVICE) as year,
         COUNT(distinct patient_id) as pat_cnt, 
         COUNT(distinct claim_number) as clm_cnt
    FROM EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_3
   GROUP BY TOC1,year
   ORDER BY TOC1,year;

Query started at 01:14:18 PM India Standard Time; Query executed in 0.06 m

toc1  year  pat_cnt  clm_cnt
0       Medicaid  2015    51066    98400
1       Medicaid  2016    51390    99405
2       Medicaid  2017    49078    94263
3       Medicaid  2018    17379    31398
4   Medicare FFS  2015   466669  1082453
5   Medicare FFS  2016   478935  1131845
6   Medicare FFS  2017   440902  1029040
7   Medicare FFS  2018   159737   347766
8         Others  2015    41241    70531
9         Others  2016    49654    83862
10        Others  2017    63317   107144
11        Others  2018    23952    39830
12       comm_ma  2015   647793  1439440
13       comm_ma  2016   661465  1489510
14       comm_ma  2017   652133  1457427
15       comm_ma  2018   286098   600916
16          None  2015    46851    73774
17          None  2016    37729    62087
18          None  2017    35911    56617
19          None  2018    15257    22399

In [20]:
raven_trends_c = raven_trends_comb.pivot_table(index = ['toc1'],
                              columns = ['year'],
                              values = ['pat_cnt','clm_cnt'],
                              fill_value= 0)
raven_trends_c

clm_cnt                           pat_cnt                  \
year             2015     2016     2017    2018    2015    2016    2017   
toc1                                                                      
Medicaid        98400    99405    94263   31398   51066   51390   49078   
Medicare FFS  1082453  1131845  1029040  347766  466669  478935  440902   
Others          70531    83862   107144   39830   41241   49654   63317   
comm_ma       1439440  1489510  1457427  600916  647793  661465  652133   

                      
year            2018  
toc1                  
Medicaid       17379  
Medicare FFS  159737  
Others         23952  
comm_ma       286098

In [23]:
# Removing levels of pivoted table
raven_trends_df_c1 = pd.DataFrame(raven_trends_c).copy()
raven_trends_df_c1.columns = ['_'.join(reversed(tuple(map(str, t)))) for t in raven_trends_c.columns.values]
raven_trends_df_c1 = raven_trends_df_c1.sort_index(axis=1)
raven_trends_df_c1 = raven_trends_df_c1.reset_index()
raven_trends_df_c1.head()

toc1  2015_clm_cnt  2015_pat_cnt  2016_clm_cnt  2016_pat_cnt  \
0      Medicaid         98400         51066         99405         51390   
1  Medicare FFS       1082453        466669       1131845        478935   
2        Others         70531         41241         83862         49654   
3       comm_ma       1439440        647793       1489510        661465   

   2017_clm_cnt  2017_pat_cnt  2018_clm_cnt  2018_pat_cnt  
0         94263         49078         31398         17379  
1       1029040        440902        347766        159737  
2        107144         63317         39830         23952  
3       1457427        652133        600916        286098

In [24]:
raven_trends_df_c1.to_excel('raven_trends_2108_c1.xlsx',index=False)

# CMS OP

In [21]:
%%read_sql
CREATE OR REPLACE TABLE eyepoint_cms_op AS
     SELECT DISTINCT 'cms_op' as SOURCE, 
                     CAST(PATIENT_ID AS STRING) AS PATIENT_ID,
                     CAST(CLAIM_NUMBER AS STRING) AS CLAIM_NUMBER,
                     service_date
       FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_OP1
      UNION
     SELECT DISTINCT 'cms_op' as SOURCE,
                     CAST(PATIENT_ID AS STRING),
                     CAST(CLAIM_NUMBER AS STRING),
                     service_date
       FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_OP_2016
      UNION
     SELECT DISTINCT 'cms_op' as SOURCE,
                     CAST(PATIENT_ID AS STRING),
                     CAST(CLAIM_NUMBER AS STRING),
                     service_date
       FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_OP_2015;

Query started at 07:33:27 PM India Standard Time; Query executed in 0.07 m

status
0  Table EYEPOINT_CMS_OP successfully created.

In [25]:
%%read_sql cms_op_trends 
    SELECT SOURCE,
           YEAR(service_date) as year,
           COUNT(DISTINCT PATIENT_ID) AS PAT_CNT,
           COUNT(DISTINCT CLAIM_NUMBER) AS CLM_CNT
      FROM eyepoint_cms_op
     GROUP BY SOURCE,year
     ORDER BY year DESC;

Query started at 07:34:39 PM India Standard Time; Query executed in 0.05 m

source  year  pat_cnt  clm_cnt
0  cms_op  2017   359771   546361
1  cms_op  2016   373691   563778
2  cms_op  2015   374646   560284

In [26]:
cms_op_trends_t = cms_op_trends.pivot_table(index=['source'],
                                            columns=['year'],
                                            values = ['pat_cnt','clm_cnt'],
                                            fill_value= 0)
cms_op_trends_t

clm_cnt                 pat_cnt                
year      2015    2016    2017    2015    2016    2017
source                                                
cms_op  560284  563778  546361  374646  373691  359771

In [30]:
# Removing levels of pivoted table
cms_op_trends_df = pd.DataFrame(cms_op_trends_t).copy()
cms_op_trends_df.columns = ['_'.join(reversed(tuple(map(str, t)))) for t in cms_op_trends_t.columns.values]
cms_op_trends_df = cms_op_trends_df.sort_index(axis=1)
cms_op_trends_df = cms_op_trends_df.reset_index()
cms_op_trends_df.head()

source  2015_clm_cnt  2015_pat_cnt  2016_clm_cnt  2016_pat_cnt  \
0  cms_op        560284        374646        563778        373691   

   2017_clm_cnt  2017_pat_cnt  
0        546361        359771

In [48]:
cms_op_trends_df.to_excel('cms_op_trends.xlsx',index=False)

# CMS ASC

In [40]:
%%read_sql
CREATE OR REPLACE TABLE eyepoint_cms_asc AS
     SELECT DISTINCT 'cms_asc' as SOURCE,
                     null as PATIENT_ID,
                     sum(ALLOWED_SERVICES) as claim_count,
                     service_year
      FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_ASC1
     GROUP BY SOURCE, PATIENT_ID, service_year
     UNION
     SELECT DISTINCT 'cms_asc' as SOURCE,
                      null as PATIENT_ID,
                      sum(ALLOWED_SERVICES) as claim_count,
                      service_year
       FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_ASC_2015
      GROUP BY SOURCE, PATIENT_ID, service_year 
       ;

Query started at 07:54:34 PM India Standard Time; Query executed in 0.08 m

status
0  Table EYEPOINT_CMS_ASC successfully created.

In [41]:
%%read_sql cms_asc_trends 
    SELECT distinct SOURCE,
           service_year,
           null AS PAT_CNT,
           claim_count
      FROM eyepoint_cms_asc
     ORDER BY service_year DESC;

Query started at 07:54:39 PM India Standard Time; Query executed in 0.06 m

source service_year pat_cnt  claim_count
0  cms_asc         2016    None      1339985
1  cms_asc         2015    None      1278570

In [45]:
cms_asc_trends_t = cms_asc_trends.pivot_table(index=['source'],
                                            columns=['service_year'],
                                            values = ['claim_count'],
                                            fill_value= 0)
cms_asc_trends_t

claim_count         
service_year        2015     2016
source                           
cms_asc          1278570  1339985

In [46]:
# Removing levels of pivoted table
cms_asc_trends_df = pd.DataFrame(cms_asc_trends_t).copy()
cms_asc_trends_df.columns = ['_'.join(reversed(tuple(map(str, t)))) for t in cms_asc_trends_t.columns.values]
cms_asc_trends_df = cms_asc_trends_df.sort_index(axis=1)
cms_asc_trends_df = cms_asc_trends_df.reset_index()
cms_asc_trends_df.head()

source  2015_claim_count  2016_claim_count
0  cms_asc           1278570           1339985

In [49]:
cms_asc_trends_df.to_excel('cms_asc_trends.xlsx',index=False)

# Average claim/patient for procedure and year

## With Med_ffs

In [7]:
%%read_sql   procedure_year_clm_pat_w_medffs

  SELECT procedure,
         year(YEAR_OF_SERVICE) as year,
         COUNT(distinct claim_number) as clm_cnt,
         COUNT(distinct patient_id) as pat_cnt         
    FROM EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_2_IB_updated_1
   GROUP BY procedure,year
   ORDER BY procedure,year;

Query started at 07:17:56 PM India Standard Time; Query executed in 0.08 m

procedure  year  clm_cnt  pat_cnt
0       1341  2015     1264      811
1       1341  2016       20       19
2       1342  2015       11        6
3       1343  2015       11        8
4        136  2015        5        5
5        136  2016        1        1
6        136  2017        1        1
7        136  2018        1        1
8       1369  2015       28       19
9       1371  2015      713      499
10     65175  2015      266      178
11     65175  2016      268      202
12     65175  2017      242      172
13     65175  2018       96       78
14     66840  2015     2668     1693
15     66840  2016     2803     1786
16     66840  2017     2544     1652
17     66840  2018     1067      673
18     66850  2015     5825     3997
19     66850  2016     5495     3639
20     66850  2017     5255     3570
21     66850  2018     2419     1630
22     66852  2015     2992     2088
23     66852  2016     2934     1945
24     66852  2017     2741     1846
25     66852  2018     1078      754
26     66920  2015      464      345
27     66920  2016      404      308
28     66920  2017      313      230
29     66920  2018       81       61
30     66940  2015      999      680
31     66940  2016      894      535
32     66940  2017      771      530
33     66940  2018      247      184
34     66982  2015   203145   111598
35     66982  2016   199046   107947
36     66982  2017   183094   101288
37     66982  2018    72597    41308
38     66983  2015     1135      759
39     66983  2016     1008      703
40     66983  2017      713      513
41     66983  2018      239      169
42     66984  2015  2051393   871492
43     66984  2016  2113892   893105
44     66984  2017  2047191   875492
45     66984  2018   821960   372015
46     66986  2015     8443     5271
47     66986  2016     8737     5356
48     66986  2017     8141     5140
49     66986  2018     3438     2180
50     G9389  2015       27       17
51     G9389  2016        3        2
52     G9389  2017       43       36
53     G9389  2018       10        7
54     G9390  2015     1016      617
55     G9390  2016     1798      949
56     G9390  2017     4815     2982
57     G9390  2018     1390      810

In [8]:
procedure_year_clm_pat_w_medffs_1 = procedure_year_clm_pat_w_medffs.pivot_table(index = ['procedure'],
                              columns = ['year'],
                              values = ['pat_cnt','clm_cnt'],
                              fill_value= 0)
procedure_year_clm_pat_w_medffs_1

clm_cnt                           pat_cnt                        
year          2015     2016     2017    2018    2015    2016    2017    2018
procedure                                                                   
1341          1264       20        0       0     811      19       0       0
1342            11        0        0       0       6       0       0       0
1343            11        0        0       0       8       0       0       0
136              5        1        1       1       5       1       1       1
1369            28        0        0       0      19       0       0       0
1371           713        0        0       0     499       0       0       0
65175          266      268      242      96     178     202     172      78
66840         2668     2803     2544    1067    1693    1786    1652     673
66850         5825     5495     5255    2419    3997    3639    3570    1630
66852         2992     2934     2741    1078    2088    1945    1846     754
66920          464      404      313      81     345     308     230      61
66940          999      894      771     247     680     535     530     184
66982       203145   199046   183094   72597  111598  107947  101288   41308
66983         1135     1008      713     239     759     703     513     169
66984      2051393  2113892  2047191  821960  871492  893105  875492  372015
66986         8443     8737     8141    3438    5271    5356    5140    2180
G9389           27        3       43      10      17       2      36       7
G9390         1016     1798     4815    1390     617     949    2982     810

In [9]:
# Removing levels of pivoted table
procedure_year_clm_pat_w_medffs_2 = pd.DataFrame(procedure_year_clm_pat_w_medffs_1).copy()
procedure_year_clm_pat_w_medffs_2.columns = ['_'.join(reversed(tuple(map(str, t)))) for t in procedure_year_clm_pat_w_medffs_1.columns.values]
procedure_year_clm_pat_w_medffs_2 = procedure_year_clm_pat_w_medffs_2.sort_index(axis=1)
procedure_year_clm_pat_w_medffs_2 = procedure_year_clm_pat_w_medffs_2.reset_index()
procedure_year_clm_pat_w_medffs_2.head()

procedure  2015_clm_cnt  2015_pat_cnt  2016_clm_cnt  2016_pat_cnt  \
0      1341          1264           811            20            19   
1      1342            11             6             0             0   
2      1343            11             8             0             0   
3       136             5             5             1             1   
4      1369            28            19             0             0   

   2017_clm_cnt  2017_pat_cnt  2018_clm_cnt  2018_pat_cnt  
0             0             0             0             0  
1             0             0             0             0  
2             0             0             0             0  
3             1             1             1             1  
4             0             0             0             0

In [10]:
procedure_year_clm_pat_w_medffs_2.to_excel('procedure_year_clm_pat_w_medffs_2.xlsx',index=False)

## Without med_ffs

In [11]:
%%read_sql   procedure_year_clm_pat_wo_medffs

  SELECT procedure,
         year(YEAR_OF_SERVICE) as year,
         COUNT(distinct claim_number) as clm_cnt,
         COUNT(distinct patient_id) as pat_cnt         
    FROM EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_trends_2_IB_updated_1
  where (lower(toc) <> 'medicare ffs' 
        or toc is null) 
   GROUP BY procedure,year
   ORDER BY procedure,year
   ;

Query started at 07:29:36 PM India Standard Time; Query executed in 0.11 m

procedure  year  clm_cnt  pat_cnt
0       1341  2015      738      541
1       1341  2016       20       19
2       1342  2015        7        5
3       1343  2015        7        5
4        136  2015        5        5
5        136  2016        1        1
6        136  2017        1        1
7        136  2018        1        1
8       1369  2015       18       15
9       1371  2015      403      310
10     65175  2015      132      102
11     65175  2016      137      106
12     65175  2017      132       96
13     65175  2018       61       49
14     66840  2015     1347      890
15     66840  2016     1420      946
16     66840  2017     1374      945
17     66840  2018      583      399
18     66850  2015     3193     2301
19     66850  2016     2975     2113
20     66850  2017     3044     2166
21     66850  2018     1475     1068
22     66852  2015     1644     1253
23     66852  2016     1663     1176
24     66852  2017     1649     1195
25     66852  2018      663      490
26     66920  2015      310      246
27     66920  2016      278      226
28     66920  2017      219      171
29     66920  2018       67       53
30     66940  2015      652      463
31     66940  2016      524      377
32     66940  2017      512      365
33     66940  2018      176      135
34     66982  2015    95032    57503
35     66982  2016    90543    54266
36     66982  2017    87550    53466
37     66982  2018    39243    24509
38     66983  2015      695      515
39     66983  2016      582      440
40     66983  2017      484      354
41     66983  2018      191      134
42     66984  2015   950073   441345
43     66984  2016   963414   449636
44     66984  2017   994404   466223
45     66984  2018   457292   223702
46     66986  2015     4174     2770
47     66986  2016     4330     2836
48     66986  2017     4212     2861
49     66986  2018     2022     1370
50     G9389  2015        3        2
51     G9389  2016        2        1
52     G9389  2017        8        8
53     G9389  2018        1        1
54     G9390  2015      105       64
55     G9390  2016      255      146
56     G9390  2017      572      314
57     G9390  2018      434      274

In [12]:
procedure_year_clm_pat_wo_medffs_1 = procedure_year_clm_pat_wo_medffs.pivot_table(index = ['procedure'],
                              columns = ['year'],
                              values = ['pat_cnt','clm_cnt'],
                              fill_value= 0)
procedure_year_clm_pat_wo_medffs_1

clm_cnt                         pat_cnt                        
year         2015    2016    2017    2018    2015    2016    2017    2018
procedure                                                                
1341          738      20       0       0     541      19       0       0
1342            7       0       0       0       5       0       0       0
1343            7       0       0       0       5       0       0       0
136             5       1       1       1       5       1       1       1
1369           18       0       0       0      15       0       0       0
1371          403       0       0       0     310       0       0       0
65175         132     137     132      61     102     106      96      49
66840        1347    1420    1374     583     890     946     945     399
66850        3193    2975    3044    1475    2301    2113    2166    1068
66852        1644    1663    1649     663    1253    1176    1195     490
66920         310     278     219      67     246     226     171      53
66940         652     524     512     176     463     377     365     135
66982       95032   90543   87550   39243   57503   54266   53466   24509
66983         695     582     484     191     515     440     354     134
66984      950073  963414  994404  457292  441345  449636  466223  223702
66986        4174    4330    4212    2022    2770    2836    2861    1370
G9389           3       2       8       1       2       1       8       1
G9390         105     255     572     434      64     146     314     274

In [15]:
# Removing levels of pivoted table
procedure_year_clm_pat_wo_medffs_2 = pd.DataFrame(procedure_year_clm_pat_wo_medffs_1).copy()
procedure_year_clm_pat_wo_medffs_2.columns = ['_'.join(reversed(tuple(map(str, t)))) for t in procedure_year_clm_pat_wo_medffs_1.columns.values]
procedure_year_clm_pat_wo_medffs_2 = procedure_year_clm_pat_wo_medffs_2.sort_index(axis=1)
procedure_year_clm_pat_wo_medffs_2 = procedure_year_clm_pat_wo_medffs_2.reset_index()
procedure_year_clm_pat_wo_medffs_2.head()

procedure  2015_clm_cnt  2015_pat_cnt  2016_clm_cnt  2016_pat_cnt  \
0      1341           738           541            20            19   
1      1342             7             5             0             0   
2      1343             7             5             0             0   
3       136             5             5             1             1   
4      1369            18            15             0             0   

   2017_clm_cnt  2017_pat_cnt  2018_clm_cnt  2018_pat_cnt  
0             0             0             0             0  
1             0             0             0             0  
2             0             0             0             0  
3             1             1             1             1  
4             0             0             0             0

In [17]:
procedure_year_clm_pat_wo_medffs_2.to_excel('procedure_year_clm_pat_wo_medffs_2.xlsx',index=False)